In [1]:
import sys
import mysql.connector as db
import os
import math
import numpy as np
import matplotlib.pyplot as plt
import UncertaintyM as unc
import warnings
import pandas as pd

# base_dir = os.path.dirname(os.path.realpath(__file__))
# pic_dir = f"{base_dir}/pic/unc"
# if not os.path.exists(pic_dir):
#     os.makedirs(pic_dir)

local          = False
job_id         = True

# data_list  = ["parkinsons","vertebral","breast","climate", "ionosphere", "blod", "bank", "QSAR", "spambase"] 
# data_list = ["parkinsons","vertebral","climate", "ionosphere", "blod"]
data_list = ["parkinsons"]
modes     = "eat"

df_res_list = []
for data in data_list:
    data_df = pd.DataFrame(columns=["job_id", "Method", "Parameter", "Epistemic", "e_sd", "Aleatoric", "a_sd", "Total", "t_sd"])
    data_df.set_index('job_id', inplace=True)
    print(f"-------------------------------------------------------------------------------------- {data}")
    # prameters ############################################################################################################################################

    run_name   = "s_delta_set24"
    # query       = f"SELECT results, id , prams, result_type FROM experiments Where task='unc' AND dataset='Jdata/{data}' AND run_name='{run_name}'"
    query       = f"SELECT results, id , prams, result_type FROM experiments Where task='unc' AND id=5604 OR id=5712 OR id=5703"

    ########################################################################################################################################################
    
    # get input from command line
    mydb = db.connect(host="131.234.250.119", user="root", passwd="uncertainty", database="uncertainty")
    mycursor = mydb.cursor()
    mycursor.execute(query) # run_name ='uni_exp'
    results = list(mycursor.fetchall())
    jobs = []
    for job in results:
        jobs.append(job)

    plot_list = []

    for job in jobs:
        dir = job[0]
        if dir[0] == ".":
            dir = base_dir + dir[1:]
        if local:
            dir = f"/home/mhshaker/Projects/Database/DB_files/job_{job[1]}"
            isFile = os.path.isdir(dir)
            if not isFile:
                print("[Error] file does not exist")
                print(dir)
                exit()
        plot_list.append(job[1])

        res_dir = dir + "/res"
        if not os.path.exists(res_dir):
            os.makedirs(res_dir)

        job_id_df = 0
        unc_method_df = "UNKNOWN"
        param_df = 0

        method_line = ""
        for mode_index, mode in enumerate(modes):
            dir_prob = dir + "/prob"
            dir_p = dir + "/p"
            dir_l = dir + "/l"
            dir_mode = dir + "/" + mode

            legend = ""
            if method_line == "":
                if job_id:
                    method_line += " " + str(job[1])
                    job_id_df = job[1]

                for text in job[3:]:
                    method_line += " " +str(text)
                    unc_method_df = str(text)


            # get the list of file names
            file_list = []
            for (dirpath, dirnames, filenames) in os.walk(dir_mode):
                file_list.extend(filenames)


            # read every file and append all to "all_runs"
            all_runs_prob = []
            for f in file_list:
                run_result = np.loadtxt(dir_prob+"/"+f)
                all_runs_prob.append(run_result)

            all_runs_unc = []
            for f in file_list:
                run_result = np.loadtxt(dir_mode+"/"+f)
                all_runs_unc.append(run_result)

            all_runs_p = []
            for f in file_list:
                run_result = np.loadtxt(dir_p+"/"+f)
                all_runs_p.append(run_result)

            all_runs_l = []
            for f in file_list:
                run_result = np.loadtxt(dir_l+"/"+f)
                all_runs_l.append(run_result)

            AUROC_mean, AUROC_std = unc.roc(all_runs_prob, all_runs_p,all_runs_l,all_runs_unc)
            method_line += f"   {mode} {AUROC_mean:.4f} +- {AUROC_std:.2f}"
            # print(f">>>>>>>>>>> {job_id_df} mode {mode}")
            if mode== "e":
                data_df.loc[job_id_df] = {"Method":unc_method_df, "Parameter": param_df, "Epistemic":AUROC_mean, "e_sd":AUROC_std}
            elif mode== "a":
                data_df.loc[job_id_df, ["Aleatoric"]] = AUROC_mean
                data_df.loc[job_id_df, ["a_sd"]] = AUROC_std
            elif mode== "t":
                data_df.loc[job_id_df, ["Total"]] = AUROC_mean
                data_df.loc[job_id_df, ["t_sd"]] = AUROC_std
            np.savetxt(f"{res_dir}/{mode}_ROC.txt", np.array([AUROC_mean, AUROC_std]))
        # print(method_line)
    # print("------------------------------------")
    data_df["Aleatoric"] = pd.to_numeric(data_df["Aleatoric"], downcast="float")
    data_df["a_sd"] = pd.to_numeric(data_df["a_sd"], downcast="float")
    data_df["Total"] = pd.to_numeric(data_df["Total"], downcast="float")
    data_df["t_sd"] = pd.to_numeric(data_df["t_sd"], downcast="float")
    print(data_df.head())
    df_res_list.append(data_df)
    


-------------------------------------------------------------------------------------- parkinsons
       Method Parameter  Epistemic      e_sd  Aleatoric      a_sd     Total  \
job_id                                                                        
5604    set18         0   0.861989  0.090318   0.866895  0.093175  0.866957   
5703     bays         0   0.843376  0.116060   0.869731  0.133187  0.870721   
5712    set19         0   0.670429  0.150555   0.866957  0.093786  0.866957   

            t_sd  
job_id            
5604    0.093617  
5703    0.116813  
5712    0.093617  


In [2]:
df_res_list[0].style.highlight_max(color = 'gray', axis = 0)

,Method,Parameter,Epistemic,e_sd,Aleatoric,a_sd,Total,t_sd
job_id,,,,,,,,
5604,set18,0,0.861989,0.090318,0.866895,0.093175,0.866957,0.093617
5703,bays,0,0.843376,0.116060,0.869731,0.133187,0.870721,0.116813
5712,set19,0,0.670429,0.150555,0.866957,0.093786,0.866957,0.093617


In [6]:
def highlight_max(s):
    # print("------------------------------------")
    # print(s)
    if s.dtype == np.object:
        is_max = [False for _ in range(s.shape[0])]
    else:
        if s.name == "e_sd" or s.name == "a_sd" or s.name == "t_sd":
            is_max = s == s.min()
        else:
            is_max = s == s.max()
    return ['background: gray' if cell else '' for cell in is_max]
  
df_res_list[0].style.apply(highlight_max)


/tmp/ipykernel_636524/2057231685.py:6: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if s.dtype == np.object:


,Method,Parameter,Epistemic,e_sd,Aleatoric,a_sd,Total,t_sd
job_id,,,,,,,,
5604,set18,0,0.861989,0.090318,0.866895,0.093175,0.866957,0.093617
5703,bays,0,0.843376,0.116060,0.869731,0.133187,0.870721,0.116813
5712,set19,0,0.670429,0.150555,0.866957,0.093786,0.866957,0.093617
